In [ ]:
from transformers import pipeline

# change model path to the path u store ur model
model_path = './intent_recognition_model'
classifier = pipeline('text-classification', model=model_path,max_length=512)

# change text to what you want to classify
text = "I recommend lenovo legion 5 for its capability of gaming and computation power"
classifier(text)

# first output indicates whether text is recommending a product, second output indicates the confidence of prediction